In [1]:
import os
import requests
import pandas as pd
import numpy as np

alphavantage_api_key = open("alphavantage_api_key.txt").read()
ticker = "afrm".upper()

# Calls the API to get historical stock data
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=full&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

# Calls the API to get earnings data
url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={alphavantage_api_key}'
r = requests.get(url)
earnings = r.json()

# Yahoo Finance API to get earnings dates
#yfinance_earnings = yf.Ticker(f'{ticker}').get_earnings_dates(limit=30)


In [20]:
##############
# Stock Data #
##############

# Extract the time series data
time_series = data['Time Series (Daily)']

# Convert the time series data into a DataFrame
df = pd.DataFrame.from_dict(time_series, orient='index')

# Convert the index to datetime
df.index = pd.to_datetime(df.index)

# Convert the columns to numeric
df = df.apply(pd.to_numeric)

df = df.reset_index().rename(columns={'index': 'tradingDate'})

#################
# Earnings Data #
#################

# Extract annual earnings data
annual_earnings = earnings['annualEarnings']

# Convert annual earnings data into a DataFrame
annual_earnings_df = pd.DataFrame(annual_earnings)

# Extract quarterly earnings data
quarterly_earnings = earnings['quarterlyEarnings']

# Convert quarterly earnings data into a DataFrame
quarterly_earnings_df = pd.DataFrame(quarterly_earnings)

last_earnings = quarterly_earnings_df[["reportedDate","reportTime"]].head(30)

# Function to adjust the date based on reportTime
def adjust_date(row):
   if row['reportTime'] == 'post-market':
       return df[df["tradingDate"] > row['reportedDate']].min(axis=0)["tradingDate"]
   elif row['reportTime'] == 'pre-market':
       return df[df["tradingDate"] < row['reportedDate']].max(axis=0)["tradingDate"]
   else:
       return row['reportedDate']

last_earnings['reportedDate'] = pd.to_datetime(last_earnings['reportedDate'])

# Apply the function to create a new column with adjusted dates
# Last earnings is used to determine how the stock moved historically around earnings
last_earnings['adjustedDate'] = last_earnings.apply(adjust_date, axis=1)

#############
# merged_df #
#############

#merged_df joins the first set of earnings data
merged_df = pd.merge(last_earnings, df, left_on='reportedDate', right_on="tradingDate", how='left')
columns_to_drop = ['tradingDate','5. volume']
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df = merged_df.rename(columns={
   '1. open': 'report open',
   '2. high': 'report high',
   '3. low': 'report low',
   '4. close': 'report close'
})

###############
# merged_2_df #
###############

#merged_2_df is the final joined dataset that combines the first set of earnings data with the second set of earnings data
merged_2_df = pd.merge(merged_df, df, left_on='adjustedDate', right_on="tradingDate", how='left')

merged_2_df.drop(columns=columns_to_drop, inplace=True)

merged_2_df = merged_2_df.rename(columns={
   '1. open': 'adjusted open',
   '2. high': 'adjusted high',
   '3. low': 'adjusted low',
   '4. close': 'adjusted close'
})

################
# calculations #
################

# used to calculate the difference between the high and low depending is earnings was pre-market or post-market
# and if the stock went up or down

# Initialize a new column with NaN values to calculate the difference between high and low
merged_2_df['diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   #Post-market
   if row['reportTime'] == 'post-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted low'] - row['report high'],2)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted high'] - row['report low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan
   #Pre-market
   elif row['reportTime'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report low'] - row['adjusted high'],2)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report high'] - row['adjusted low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan

# Initialize a new column with NaN values for the percentage calculation
merged_2_df['percent_diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   if row['reportTime'] == 'post-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round((row['adjusted low'] / row['report high'])-1,3)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['adjusted high'] / row['report low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan
   elif row['reportTime'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report low'] / row['adjusted high']-1,3)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report high'] / row['adjusted low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan

merged_2_df.sort_values(by="percent_diff", ascending=False)


,reportedDate,reportTime,adjustedDate,report open,report high,report low,report close,adjusted open,adjusted high,adjusted low,adjusted close,diff,percent_diff
13,2022-05-12,post-market,2022-05-13,14.140,19.3700,13.6401,18.04,24.8300,25.3200,21.2100,23.71,11.68,0.856
16,2021-09-09,post-market,2021-09-10,88.000,92.2900,86.7600,92.06,111.1000,126.4600,105.6000,123.70,39.70,0.458
4,2024-08-28,post-market,2024-08-29,32.230,32.5100,30.4601,31.58,39.1100,42.8700,37.5200,41.66,12.41,0.407
7,2023-11-08,post-market,2023-11-09,21.510,22.1897,20.2000,21.76,24.9700,27.1566,24.4300,24.86,6.96,0.344
8,2023-08-24,post-market,2023-08-25,14.550,14.5800,13.7400,13.81,15.0900,18.3200,15.0000,17.79,4.58,0.333
2,2025-02-06,post-market,2025-02-07,63.120,63.3599,60.5600,61.75,69.9000,76.8800,68.3000,75.22,16.32,0.269
18,2021-01-14,post-market,2021-01-15,103.500,137.9800,101.0000,114.94,123.0540,127.2400,107.0000,117.00,26.24,0.260
15,2021-11-10,post-market,2021-11-11,146.930,149.0000,132.1600,133.53,165.2501,166.5100,147.5000,151.83,34.35,0.260
5,2024-05-08,post-market,2024-05-09,35.500,37.8800,31.0700,31.58,32.6100,34.8900,32.3200,34.81,3.82,0.123
1,2025-05-08,pre-market,2025-05-07,53.575,55.7600,53.3003,54.26,50.7350,52.2000,50.0277,51.54,5.73,0.115
